In [1]:
# from google.colab import drive
# drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [2]:
%cd drive/MyDrive/Dersler/HWs/NLP/HW2/
%ls

/content/drive/MyDrive/Dersler/HWs/NLP/HW2
Dataset/  Glove/  HW2.ipynb  nlp2020-hw2/  Saved/


In [3]:
from tqdm.auto import tqdm
import torch

In [4]:
import json, os
json_data = "dev"

with open('Dataset/train.json', 'r') as json_file:
    train_raw = json.load(json_file)
with open('Dataset/dev.json', 'r') as json_file:
    dev_raw = json.load(json_file)
with open('Dataset/test.json', 'r') as json_file:
    test_raw = json.load(json_file)

In [5]:
print(len(train_raw))
print("words\n", train_raw["0"]["words"]) # input
print("lemmas\n", train_raw["0"]["lemmas"]) # alternative input
print("pos_tags\n", train_raw["0"]["pos_tags"])
print("dependency_heads\n", train_raw["0"]["dependency_heads"])
print("dependency_relations\n", train_raw["0"]["dependency_relations"])
print("predicates\n", train_raw["0"]["predicates"])
print("roles\n", train_raw["0"]["roles"]) # Find this

39279
words
 ['In', 'an', 'Oct.', '19', 'review', 'of', '``', 'The', 'Misanthrope', "''", 'at', 'Chicago', "'s", 'Goodman', 'Theatre', '(', '``', 'Revitalized', 'Classics', 'Take', 'the', 'Stage', 'in', 'Windy', 'City', ',', "''", 'Leisure', '&', 'Arts', ')', ',', 'the', 'role', 'of', 'Celimene', ',', 'played', 'by', 'Kim', 'Cattrall', ',', 'was', 'mistakenly', 'attributed', 'to', 'Christina', 'Haag', '.']
lemmas
 ['in', 'an', 'oct.', '19', 'review', 'of', '``', 'the', 'misanthrope', "''", 'at', 'chicago', "'s", 'goodman', 'theatre', '-lrb-', '``', 'revitalize', 'classics', 'take', 'the', 'stage', 'in', 'windy', 'city', ',', "''", 'leisure', '&', 'arts', '-rrb-', ',', 'the', 'role', 'of', 'celimene', ',', 'play', 'by', 'kim', 'cattrall', ',', 'be', 'mistakenly', 'attribute', 'to', 'christina', 'haag', '.']
pos_tags
 ['IN', 'DT', 'NNP', 'CD', 'NN', 'IN', '``', 'DT', 'NN', "''", 'IN', 'NNP', 'POS', 'NNP', 'NNP', '(', '``', 'VBN', 'NNS', 'VBP', 'DT', 'NN', 'IN', 'NNP', 'NNP', ',', "''", '

In [ ]:
word2id = {}
id2word = {}

word2id["<PAD>"] = 0
id2word[0] = "<PAD>"
word2id["<UNK>"] = 1
id2word[1] = "<UNK>"
word2id["<VERB>"] = 2
id2word[2] = "<VERB>"

def tokenizer(data, name):
  count=len(word2id)
  for lines in data:
    for word in data[lines][name]:
      if word != "_":
        
        try:
          id = word2id[word]
          # id2word[id] = word
        except:
          word2id[word] = count
          id2word[count] = word
          count += 1

typeOfInput = "words" # words or lemmas

tokenizer(train_raw, typeOfInput)
# tokenizer(dev_raw, typeOfInput)
tokenizer(train_raw, "predicates")
# tokenizer(dev_raw, "predicates")

print("Vocabulary size:", len(word2id))
print("Word to id:\n",word2id)
print("Word id to word:\n",id2word)

In [ ]:
def indexer(data, name):
  count = 0
  unk_count = 0
  indexed_sentences = []
  for lines in data:
    
    for NoOfRoles in range(len(data[lines]["roles"])):
      verb_count = 0
      indexed_words = []
      for word_no in range(len(data[lines][name])):

        predicate = data[lines]["predicates"][word_no]
        word = data[lines][name][word_no]
        
        verb_flag = False
        if predicate != "_":
          verb_count += 1
          if verb_count == NoOfRoles+1:
            id = word2id[predicate]
            # id = word2id["<VERB>"]
            verb_flag = True
          
        if not verb_flag:
          try:
            id = word2id[word]
          except:
            unk_count += 1
            id = word2id["<UNK>"]

        indexed_words.append(id)
      indexed_sentences.append(indexed_words)

  if unk_count>0:
    print("Number of unknown word:", unk_count)

  return indexed_sentences

train_indexed_x = indexer(train_raw, typeOfInput)
dev_indexed_x = indexer(dev_raw, typeOfInput)
print("\nTest dataset,")
test_indexed_x = indexer(test_raw, typeOfInput)

print("\nExample 'word index' - 'word' pairs")
for word in train_indexed_x[0]:
  print(word, "-", id2word[word])

In [ ]:
label2id = {}
id2label = {}
label2id["<PAD>"] = 0
id2label[0] = "<PAD>"

def label_vocab(data, name):
  for lines in data:
    for role_set in data[lines][name]:
      for role in data[lines][name][role_set]:
        try:
          count=len(label2id)
          id = label2id[role]
          id2label[id] = role
        except:
          label2id[role] = count
          id2label[count] = role
          count += 1

label_vocab(train_raw, "roles")
print("Number of label:", len(label2id))

print("Label to id:", label2id)
print("Id to label:", id2label)

def label_indexer(data, max_length=0):
  count = 0
  unk_count = 0
  indexed_sentences = []
  for lines in data:
    for set_roles in data[lines]["roles"]:
      indexed_words = []

      for role in data[lines]["roles"][set_roles]:        
        id = label2id[role]
        indexed_words.append(id)
      indexed_sentences.append(indexed_words)

  if unk_count>0:
    print("\nNumber of unknown label:", unk_count)

  return indexed_sentences

train_indexed_y = label_indexer(train_raw)
dev_indexed_y = label_indexer(dev_raw)
test_indexed_y = label_indexer(test_raw)

In [ ]:
for lines in train_raw:
  for set_roles in train_raw[lines]["roles"]:
      for role in train_raw[lines]["roles"][set_roles]:
        print(role)

In [ ]:
a = len(max(train_indexed_y, key=len))
b = len(max(dev_indexed_y, key=len))
c = len(max(test_indexed_y, key=len))
max_length = max(a, b, c)
print("Max sentence length =", max_length)

In [ ]:
def padding_opt(dataset, max_length=max_length):
  for line in dataset:
    while len(line) < max_length:
      line.append(word2id["<PAD>"])
  return dataset

train_padded_x = padding_opt(train_indexed_x)
train_padded_y = padding_opt(train_indexed_y)

dev_padded_x = padding_opt(dev_indexed_x)
dev_padded_y = padding_opt(dev_indexed_y)

test_padded_x = padding_opt(test_indexed_x)
test_padded_y = padding_opt(test_indexed_y)

In [ ]:
def build_dataset(inputs, outputs, device="cuda"):
  dataset = []
  for i in tqdm(range(len(inputs))):
    x = inputs[i]
    y = outputs[i]
    
    element = {"inputs": torch.LongTensor(x).to(device), "outputs": torch.LongTensor(y).to(device)}
    dataset.append(element)
  return dataset

train_dataset = build_dataset(train_padded_x, train_padded_y)
valid_dataset = build_dataset(dev_padded_x, dev_padded_y)
test_dataset = build_dataset(test_padded_x, test_padded_y)

In [ ]:
import time
import csv
from tqdm.auto import tqdm
import numpy as np
import torch
from torch import nn
from torch.utils.data import DataLoader
import torch.optim as optim
import matplotlib.pyplot as plt
import pandas as pd
from sklearn.metrics import confusion_matrix
from sklearn.metrics import f1_score
import seaborn as sn
import numpy as np

In [ ]:
train_dataset = DataLoader(train_dataset, batch_size=32, shuffle=True)
valid_dataset = DataLoader(valid_dataset, batch_size=32)
test_dataset = DataLoader(test_dataset, batch_size=32)

In [ ]:
%cd /content/drive/My Drive/HWs/NLP/HW2/
embeddings = {}
path = path = "Glove/"
name = "glove.6B.100d.txt"
with open(path + name, 'r') as f:
  for line in f:
    values = line.split()
    word = values[0] # first element is the word
    vector = np.asarray(values[1:], "float32") # rest is vector elements
    embeddings[word] = vector

pre_trained_emb = []


for word in word2id:
  
  try:
    x = embeddings[word]  
    
  except:
    x = np.random.rand(100)

  pre_trained_emb.append(torch.LongTensor(x))

In [ ]:
class SimpleModel(nn.Module):
    def __init__(self, vocab_length=len(word2id), embedding=pre_trained_emb, num=2, dropout=0.5):
        super(SimpleModel, self).__init__()
        self.word_embedding = nn.Embedding(vocab_length, 100, padding_idx=word2id['<PAD>']) # vocab size, embedding dimension size
        if embedding:
          print("pretrained embedding!")
          self.word_embedding.weight.data.copy_(torch.stack(embedding))

        self.lstm = nn.LSTM(100, # embedding dimension
                            100, # hidden dimension
                            bidirectional=True, 
                            num_layers=num, # number of lstm layer
                            dropout = dropout, 
                            batch_first=True)
        
        lstm_output_dim = 200 # "hidden dimension x 2" for bidirectional
        self.dropout = nn.Dropout(dropout)
        self.classifier = nn.Linear(lstm_output_dim, len(label2id)) # lstm output dimension, number of the labels in the dataset

    def forward(self, x):
        embeddings = self.word_embedding(x)
        embeddings = self.dropout(embeddings)
        o, (h, c) = self.lstm(embeddings)
        o = self.dropout(o)
        output = self.classifier(o)
        return output

In [ ]:
class Trainer():
  def __init__(self, model: nn.Module, loss_function, optimizer, 
               label_vocab: label2id):
    
    self.model = model
    self.loss_function = loss_function
    self.optimizer = optimizer

    self.label_vocab = label_vocab
        
  def train(self, train_dataset, 
            valid_dataset, 
            epochs:int=1):

    assert epochs > 0 and isinstance(epochs, int)

    train_loss = 0.0
    max_f1 = 0
    for epoch in range(epochs):
      
      epoch_loss = 0.0
      self.model.train()
      print("Epoch:", epoch+1, "/", epochs)
      for step, data in enumerate(tqdm(train_dataset)):    
        self.optimizer.zero_grad()

        inputs = data['inputs']
        labels = data['outputs']

        preds = self.model(inputs)
        preds = preds.view(-1, preds.shape[-1])
        labels = labels.view(-1)
        
        sample_loss = self.loss_function(preds, labels)
        sample_loss.backward()
        self.optimizer.step()

        epoch_loss += sample_loss.tolist()

      avg_epoch_loss = epoch_loss / len(train_dataset)
      train_loss += avg_epoch_loss
      valid_loss = self.evaluate(valid_dataset)

      print("Train loss = {:0.4f}, Valid loss = {:0.4f}".format(avg_epoch_loss, valid_loss))

      f1 = get_f1(simple, test=test_dataset)

      if max_f1 < f1:
        max_f1 = f1
        torch.save(self.model.state_dict(), "Saved/" + str(f1))
        print("Model saved!,",max_f1)

      loss_x.append(avg_epoch_loss)
      loss_y.append(valid_loss)

    avg_epoch_loss = train_loss / epochs
     
    
    return avg_epoch_loss


  def evaluate(self, valid_dataset):

    valid_loss = 0.0
    self.model.eval()
    with torch.no_grad():
      for data in valid_dataset:
        inputs = data['inputs']
        labels = data['outputs']

        preds = self.model(inputs)
        preds = preds.view(-1, preds.shape[-1])
        labels = labels.view(-1)
        sample_loss = self.loss_function(preds, labels)
        valid_loss += sample_loss.tolist()
    
    return valid_loss / len(valid_dataset)

  def predict(self, x):

    self.model.eval()
    with torch.no_grad():
      logs = self.model(x)
      preds = torch.argmax(logs, -1)
      return logs, preds

In [ ]:
def get_f1(model:nn.Module, test:DataLoader, get_preds= False):
    predictions = list()
    labels = list()
    for data in test:
        x = data["inputs"]
        pred = model(x)
        pred = torch.argmax(pred, -1).view(-1)

        y = data["outputs"]
        label = y.view(-1)
        valid_indices = label != 0
        
        valid_predictions = pred[valid_indices]
        valid_labels = label[valid_indices]
        
        predictions.extend(valid_predictions.tolist())
        labels.extend(valid_labels.tolist())
        
    id_precision, id_recall, id_f1 = argument_identification(predictions, labels)
    cl_precision, cl_recall, cl_f1 = argument_classification(predictions, labels)

    # print("Argument Identification - f1 score:", id_f1)
    # print("Argument disambiguation - f1 score:", cl_f1)
    print("Argument Identification - f1 score = {:0.4f}\nArgument Disambiguation - f1 score = {:0.4f}\n".format(id_f1, cl_f1))
    if get_preds:
      return predictions, labels, cl_f1
    
    return cl_f1

def argument_identification(pred, gold):
  tp, fp, fn = 0, 0, 0
  excluded = label2id["_"]
  for p, g in zip(pred, gold):    
    if p != excluded and g != excluded:
      tp += 1
    elif p != excluded and g == excluded:
      fp += 1
    elif p == excluded and g != excluded:
      fn += 1
  precision = tp / (tp + fp)
  recall = tp / (tp + fn)
  f1 = 2 * (precision * recall) / (precision + recall)
  return precision, recall, f1

def argument_classification(pred, gold):
  tp, fp, fn = 0, 0, 0
  excluded = label2id["_"]
  for p, g in zip(pred, gold):
    if p != excluded and g != excluded:      
      if p == g:
        tp += 1
      else:
        fp += 1
        fn += 1
    elif p != excluded and g == excluded:
      fp += 1
    elif p == excluded and g != excluded:
      fn += 1
  precision = tp / (tp + fp)
  recall = tp / (tp + fn)
  f1 = 2 * (precision * recall) / (precision + recall)
  return precision, recall, f1

In [ ]:
# Save:

# torch.save(model.state_dict(), PATH)
# Load:

# model = TheModelClass(*args, **kwargs)
# model.load_state_dict(torch.load(PATH))
# model.eval()

In [ ]:
simple = SimpleModel().cuda()
simple_trainer = Trainer( 
  model=simple,
  loss_function=nn.CrossEntropyLoss(ignore_index=word2id['<PAD>']),
  optimizer=optim.Adam(simple.parameters(), lr=0.003),
  label_vocab=label2id
)

loss_x = []
loss_y = []
simple_trainer.train(train_dataset, valid_dataset, 20)

plt.plot(loss_x, "-b", loss_y , "-r")
print("Testing on test dataset,\n")
pred_y, gold_y, _ = get_f1(simple, test_dataset, get_preds=True)
data = confusion_matrix(pred_y, gold_y)
print(data)

In [ ]:
simple.eval()
simple(encoded_a)

In [ ]:
def encode(data, vocab, device="cuda", max=max_length):
  length_sentences = []
  encoded_sentences = []
  for sentence in data:    
    encoded_words = []
    for word in sentence:
      try:
        element = vocab[word]
      except:
        element = vocab["<UNK>"]
      encoded_words.append(element)
    length_sentences.append(len(sentence))
    while len(encoded_words) < max:
      encoded_words.append(vocab["<PAD>"])
    encoded_sentences.append(torch.LongTensor(encoded_words).to(device))
  return torch.stack(encoded_sentences), length_sentences

def decode(data, labels, length):
  decoded_sequences = []
  for seq_index in range(len(data)):
    decoded_labels = []
    for word_lengths in range(length[seq_index]):
      sequence = data[seq_index]
      label_id = sequence[word_lengths]
      for key in labels.keys():
        if label_id == labels[key]:
          decoded_labels.append(key)
    decoded_sequences.append(decoded_labels)
  return decoded_sequences

a = [["The", "cat", "ate", "the", "fish", "."]]

encoded_a, lengths = encode(a, word2id)
logits = model(encoded_a)
predictions = torch.argmax(logits, -1)
print(predictions)
pred = decode(predictions, id2label, lengths)
print(pred)


In [ ]:
for x_id in range(len(pred[0])):
  print(a[0][x_id],":", pred[0][x_id])

In [ ]:

pred_y, gold_y, _ = get_f1(simple, encoded_a, get_preds=True)

In [ ]:
for x_id in range(len(pred[0])):
  print(a[0][x_id],":", pred[0][x_id])

In [ ]:
cm_hm = pd.DataFrame(data, columns=np.unique(gold_y), index = np.unique(gold_y))
cm_hm.index.name = 'Actual'
cm_hm.columns.name = 'Predicted'
plt.figure(figsize = (20,14))

sn.heatmap(cm_hm, cmap="gist_heat_r", annot=True, annot_kws={"size": 16}, fmt="d")

In [ ]:
path = "Saved/0.801840490797546"
model = SimpleModel().cuda()
model.load_state_dict(torch.load(path))
model.eval()
f1 = get_f1(model, test_dataset)
print(model)
print("The model I used, f1:", f1)

In [ ]:
pred_y, gold_y, _ = get_f1(model, test_dataset, get_preds=True)
encoded_a, lengths = encode(a, word2id)
logits = model(encoded_a)
predictions = torch.argmax(logits, -1)
print(predictions)
pred = decode(predictions, id2label, lengths)
print(pred)